# Data Preperation and Preprocessing 

to obtain paired data of tokenised groundtruth of ASAP dataset, and quantised MIDI input from PM2S Quantisation

### Utility Midi to Text Function

In [ ]:
# utility function
def midi_to_txt(file):
    filename = file
    midx = MidiFile(filename, clip=True)
    f = open(filename + ".txt", "w") # overwrites any existing files of same name

    for track in midx.tracks:
        for msg in track:
            if msg.type == 'set_tempo':
                # calculate actual tempo in quarter notes per minute
                actual_tempo = int(60000000/msg.tempo)

                f.write(str(msg) + ' ----------- ' + str(actual_tempo))
                f.write("\n")

            else:
                f.write(str(msg))
                f.write("\n")

    f.close()
    print('written ' + filename + ".txt")

## Cleaning MIDI (Preprocess to Include NoteOff Events)

Some MIDI Controllers send note on with velocity=0 to indicate Note Off, but presence of Note On makes the output of PM2S very messy.


**UPDATE: Prob not needed**

In [ ]:
# from mido import Message, MidiFile, MidiTrack, MetaMessage
# import os, mido

# # Get one MIDI recording from the ASAP dataset
# file = "../ASAP_Dataset/Bach_846_perfShi05M.mid"
# filename = os.path.basename(os.path.normpath(file))

In [ ]:
# mid = MidiFile(file, clip=True) # clip=True clips the velocity of all notes to 127 if they are higher
# mid2 = MidiFile()

# # The two files need to have the same ticks per beat, very important
# mid2.ticks_per_beat = mid.ticks_per_beat

# for track in mid.tracks:
#     track2 = MidiTrack()
#     mid2.tracks.append(track2)
#     for msg in track:
#         if msg.type == 'note_on' and msg.velocity == 0: # ie essentially a note off
#             track2.append(Message('note_off', channel=msg.channel, note=msg.note, velocity=msg.velocity, time=msg.time))
            
#         else: # other types of msg
#             track2.append(msg)
    
    
# mid2.save('../ASAP_Dataset/note_off_'+ filename)
# print('DONE cleaning (adding Note Off) for ' + 'note_off_'+ filename)
# midi_to_txt('../ASAP_Dataset/note_off_'+ filename)

## Quantising MIDI Input (PM2S)


Not using this code due to errors in output --> Using Author's Code instead, which outputs a MIDI file, which will then be tokenised

In [ ]:
# # ref: https://github.com/cheriell/PM2S/blob/main/demo.ipynb 
# from PM2S.pm2s.features.quantisation import RNNJointQuantisationProcessor

# # Get one MIDI recording from the ASAP dataset
# midi_recording = "../ASAP_Dataset/Bach_846_perfShi05M.mid"
# # midi_recording = "../ASAP_Dataset/twinkle-twinkle-little-star.mid"
# # midi_recording = "../ASAP_Dataset/MAPS_MUS-pathetique_1_ENSTDkCl.mid"

# # Create a quantisation processor
# processor = RNNJointQuantisationProcessor()

# # Process the MIDI recording to the beat predictions
# beats, onset_positions, note_values = processor.process(midi_recording)
# print('onset positions and note values in number of beats:')
# print(onset_positions[:20])
# print(note_values[:20])

## Tokenising MIDI using REMI
ref: https://miditok.readthedocs.io/en/v2.1.6/examples.html 


### REMI Quantisation Resolution
ref: https://miditok.readthedocs.io/en/latest/midi_tokenizer.html#miditok.TokenizerConfig 

In [ ]:
beat_res = {(0, 4): 8, (4, 12): 4} 

max(beat_res.values())

Q is defined as 4 * max(beat_res.values()) and beat_res has a default of {(0, 4): 8, (4, 12): 4}

Maximum resolution of 1 bar will be obtained by 4* max(beat_res().values)

So by default it is 4 * 8 = 32


**To replicate ScoreTransformer Q = 24, need to change max(...) = 6 instead**

Yes 32 makes more sense

Or does it?

In [ ]:
from miditok import REMI, REMIPlus, TokenizerConfig  # here we choose to use REMI

# Get one MIDI recording from the ASAP dataset
midi_recording = "../ASAP_Dataset/Bach_846_perfShi05M.mid"

# Our parameters
TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": {(0, 4): 6}, # this is Q=24 for Duration token
    "nb_velocities": 32,
    "special_tokens": ["PAD", "BOS", "EOS", "MASK"],
    "use_chords": False,
    "use_rests": True,
    "use_tempos": False,
    "use_time_signatures": False,
    "use_programs": False,
    "nb_tempos": 32,  # nb of tempo bins
    "tempo_range": (40, 250),  # (min, max)
    "beat_res_rest": {(0, 4): 6},
    # "beat_res_rest": {(0, 1): 8, (1, 2): 4, (2, 12): 2},
}
config = TokenizerConfig(**TOKENIZER_PARAMS)

# Creates the tokenizer
tokenizer = REMIPlus(config)# if add attribute max_bar_embedding=5000, will increase vocab size cos each bar is now unique

In [ ]:
from miditoolkit import MidiFile

# Tokenize a MIDI file
midi = MidiFile(midi_recording)
tokens = tokenizer(midi)  # automatically detects MidiFile, paths

# keys include .tokens, .ids, .bytes (none for now), .events
print(tokens.tokens[:50])

In [ ]:
print(len(tokenizer)) # len of vocab

In [ ]:
len(tokens.tokens)

## Combining PM2S and REMI+

As outlined in ScoreTransformer paper, only tokens used are: 
**bar, beat, position (pos), note pitch (note), note duration (len) tokens.**


New Duration_token of a note will be note values, just replace with the PM2S values accordingly

For Beats:
first need to match note to note.

Then change POS to the decimal * 24 (to get a int)

Then everytime theres an increment of 1 beat, add a “beat” token, and decrease the POS tokens by the accumulated number of beats


### Loading Pickle Files generated by Pm2S

2 pickle files, one for duration in terms of beats, one for onset times, also in terms of beats

### Note Duration Pm2S Q=24

In [ ]:
import pickle
with open("../ASAP_Dataset/ASAP_pickle_Q=24/S_208_note_value_pickle", "rb") as fp:   # Unpickling
    v = pickle.load(fp)
    
print(v[:20])

print(len(v))

In [ ]:
for d in v:
    print(d*24)

### PM2S Onset Q=24

In [ ]:
import pickle
# load pickle file
with open("../ASAP_Dataset/ASAP_pickle_Q=24/S_208_onset_pickle", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
    
b[:20]

In [ ]:
# everything here should be a whole number cos it should be *quantised* wrt to 24
for x in b:
    decimal_part = x%1
    print(decimal_part*24)

#### Removing Tokens From REMI+

Specifically, only keeping **bar, beat, rest, position (pos), note pitch (note), note duration (len) tokens.**

Duration and pos will be modified and overriden with PM2S values

Beat will be added based on onset_musical as predicted by PM2S

In [ ]:
seq = tokens.tokens
Q = 24 # the resolution of quantisation. for pm2s and scoretransformer, it is set at 32

In [ ]:
# removing 'Tempo','Velocity','Program', 'TimeSig'
for i, elem in enumerate(list(seq)):
    token_type = elem.split('_')[0]
    if token_type in ['Tempo','Velocity','Program', 'TimeSig', 'Bar']:
        seq.remove(elem)  
#         if token_type == 'Tempo':
#             # need remove the Position token tagged to it as well
#             # No tempo tag in this specific eg
#             seq.remove(seq[i-2])  
            
print(len(seq))
print(seq[:20])

In [ ]:
# replacing the duration with values from pickle file (v[])
import numpy as np
note_index = 0
duration_index = 0
for i, elem in enumerate(list(seq)):
    token_type = elem.split('_')[0]
    if token_type == 'Duration':
        duration_index += 1
        
        # replace duration value with Pm2S output
        seq[i] = 'Duration_' + str(int(np.round(v[note_index]*Q)))
        note_index += 1
        
print(len(seq))
print(seq[:20])
print(duration_index)
print(note_index)

In [ ]:
# replacing Position and adding 'beat' before 'position' tokens 
import math
import numpy as np

beat_index = 0
new_seq = []
note_index = 0

for i, elem in enumerate(list(seq)):
    token_type = elem.split('_')[0]
    if token_type == 'Position':
        # replace value with the decimal part
        decimal_part = b[note_index] % 1
        int_part = math.floor(b[note_index]) 
        
#         if int_part == 0:
#             new_seq.append('Beat') # to cover the first beat
        
        if int_part > beat_index:
            new_seq.append('Beat')
            beat_index += 1
            
        pos_elem = 'Position_' + str(int(np.round(decimal_part * Q))) # np.round() used to avoid rounding errors
        new_seq.append(pos_elem)
        
        note_index += 1
        
    else:
        new_seq.append(elem)

new_seq.insert(0, 'Beat') # to handle first beat
print(len(new_seq))
print(new_seq[:20])

In [ ]:
print(new_seq)

In [ ]:
print(beat_index)

### Get Int Tokens from REMI Vocab


In [ ]:
# import miditok

# tokSeq = miditok.convert_sequence_to_tokseq(tokenizer, new_seq, complete_seq = False, decode_bpe=False)
tokSeq = new_seq

In [ ]:
import pickle

with open('REMI_vocab_q24.pkl', 'rb') as handle:
    b = pickle.load(handle)

In [ ]:
print(len(b))

In [ ]:
token_dict = {}

for token in sorted(b):
        token_dict[token] = len(token_dict)

In [ ]:
print(token_dict)

## Tokenising Output / Groundtruth

In [1]:
# ref: https://github.com/suzuqn/ScoreTransformer/tree/main/tokenization_tools 
from score_to_tokens import MusicXML_to_tokens 

#test_score = "../ASAP_Dataset/Bach_846_score.musicxml"
test_score = "problem_scores/haydn_sonata49-1.musicxml"
# test_score = "exported_14-3_xml_score.musicxml"

tokens = MusicXML_to_tokens(test_score)

print(len(tokens))

print(tokens[:10])

1
[[<measure number="1" width="246.38"><print><system-layout><system-margins><left-margin>20.84</left-margin><right-margin>0.00</right-margin></system-margins><top-system-distance>170.00</top-system-distance></system-layout><staff-layout number="2"><staff-distance>93.88</staff-distance></staff-layout></print><attributes><divisions>24</divisions><key><fifths>-3</fifths><mode>major</mode></key><time><beats>3</beats><beat-type>4</beat-type></time><staves>2</staves><clef number="1"><sign>G</sign><line>2</line></clef><clef number="2"><sign>F</sign><line>4</line></clef></attributes><direction placement="above"><direction-type><words font-family="Times New Roman" font-size="11" font-weight="bold" relative-y="20.00">Allegro</words></direction-type><staff>1</staff></direction><note default-x="125.37" default-y="-65.00" id="n1"><pitch><step>G</step><octave>3</octave></pitch><duration>6</duration><voice>1</voice><type>16th</type><stem>up</stem><staff>1</staff><beam number="1">begin</beam><beam nu

IndexError: list index out of range

In [ ]:
# write out
with open('tokenised_846.txt', 'w') as f:
    f.write(' '.join(tokens))

In [ ]:
# trying to detokenise the same file
from tokens_to_score import tokens_to_score

token_str = " ".join(tokens)

s = tokens_to_score(token_str)


s.write('musicxml', 'generated_score')